# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Функция для построения сводных во всех вопросах
def dependence_between(data, parameter_1, parameter_2='debt'):
    return data.groupby(parameter_1)[parameter_2].agg(['count', 'sum', 'mean'])

In [ ]:
df = dependence_between(data, 'children')
df

In [ ]:
# указываем размер визуализации
plt.rcParams['figure.figsize'] = (8, 4)

# формируем график
bar=sns.barplot(x=df.index, y='mean', data=df[['mean']], palette='dark:#4287f5', alpha=0.8)

# добавляем подписи данных
for index, value in enumerate(df['mean']):
    label = format(float(value), '.2%')
    plt.annotate(label, xy=(index-0.15, value+0.005), color='black')
    
# корректируем заголовок, оси, подписи осей
plt.title("Зависимость между количеством детей и возвратом кредита в срок\n".upper(), loc='center')
plt.xticks()
plt.yticks([])

plt.ylabel("Не вернули кредит, %")
plt.xlabel("Количество детей")

bar.spines.clear()

**Вывод:** 

Люди, у которых *количество детей равно 5*, никогда не имели задолженности по кредиту. Однако, стоит учитывать, что количество людей в выборке с количеством детей равном 5 составляет 9 человек. 

Больше всего данных имеется для категорий заемщиков без детей, с 1 и 2 детьми. Процент невозврата кредита меньше у людей, не имеющих детей. Для категорий с 1 или 2 детьми процент невозврата выше приблизительно на 2%. Это может быть связано с увеличенными тратами на детей.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df = dependence_between(data, 'family_status')

df.index = [f'{label.capitalize()}' for label in df.index]
df = df.sort_values(by='mean')
df

In [ ]:
# указываем размер визуализации
plt.rcParams['figure.figsize'] = (8, 4)

# формируем график
bar=sns.barplot(x='mean', y=df.index, data=df[['mean']], palette='dark:#4287f5', alpha=0.8)

# добавляем подписи данных
for index, value in enumerate(df['mean']):
    label = format(float(value), '.2%')
    plt.annotate(label, xy=(value+0.001, index), color='black')
    
# корректируем заголовок, оси, подписи осей
plt.title("Зависимость между семейным положением и возвратом кредита в срок\n".upper(), loc='center')
plt.xticks()
plt.yticks()
plt.xlabel("Не вернули кредит, %")
plt.ylabel("Семейное положение")
bar.spines.clear()

**Вывод:** 

Для любого семейного положения процент возврата кредита примерно одинаковый и равен от 6 до 10%.

Люди с семейным положением *"Не женат / не замужем"* чаще остальных не возвращают кредит. 

Люди с семейным положением *"Вдовец / вдова"* реже остальных не возвращают кредит. 

Люди, которые никогда не были женаты / замужем, чаще не возвращают кредиты. Возможно, есть связь с возрастом.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Немного скорректируем функцию, чтобы в графике можно было сделать сортировку по категории и вывести понятные значения 
def income_group (income):
    try: 
        if 0 <= income <= 30000:
            return pd.Series(['E', 'до 30 000'])
        elif 30001 <= income <= 50000:
            return pd.Series(['D', 'от 30 000 до 50 000'])
        elif 50001 <= income <= 200000:
            return pd.Series(['C', 'от 50 000 до 200 000'])
        elif 200001 <= income <= 1000000:
            return pd.Series(['B', 'от 200 000 до 1 000 000'])
        elif income >= 1000001:
            return pd.Series(['A', 'выше 1 000 000'])    
    except:
        pass
    
# Применим функцию и получим категории на основании дохода 
df = pd.DataFrame()
data[['total_income_category', 'income_group']] = data['total_income'].apply(income_group)

df = dependence_between(data, ['total_income_category', 'income_group'])
df = df.reset_index(0).reset_index(drop=False)
df

In [ ]:
# указываем размер визуализации
plt.rcParams['figure.figsize'] = (16, 4)

# формируем график
bar=sns.barplot(y='mean', x='income_group', data=df[['income_group', 'mean']], palette='dark:#4287f5', alpha=0.8, hue='income_group', legend=False)

# добавляем подписи данных
for index, value in enumerate(df['mean']):
    label = format(float(value), '.2%')
    plt.annotate(label, xy=(index-0.06, value+0.003), color='black')
    
# корректируем заголовок, оси, подписи осей
plt.title("Зависимость между уровнем дохода и возвратом кредита в срок\n".upper(), loc='center')
plt.yticks()
plt.yticks()

# Скорректировала подписи
plt.ylabel("Не вернули кредит, %")
plt.xlabel("Категория дохода")
bar.spines.clear();

**Вывод:** 
Для каждой категории дохода процент не возврата кредита составляет от 6 до 9%.

Больше всего данных имеется для категорий C (от 50 000 до 200 000) и B (от 200 000 до 1 000 000).

Заемщики с более низким доходом (из категории С) чаще не возвращают кредиты, чем заемщики с более высоким доходом (из категории B).

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Найдем отношение количества людей, не имеющих задолженности, к общему количеству людей каждой категории дохода
df = dependence_between(data, 'purpose_category')

df.index = [f'{label.capitalize()}' for label in df.index]
df = df.sort_values(by='mean')
df

In [ ]:
# указываем размер визуализации
plt.rcParams['figure.figsize'] = (8, 4)

# формируем график
bar=sns.barplot(x='mean', y=df.index, data=df[['mean']], palette='dark:#4287f5', alpha=0.8, hue='mean', legend=False)

# добавляем подписи данных
for index, value in enumerate(df['mean']):
    label = format(float(value), '.2%')
    plt.annotate(label, xy=(value+0.001, index), color='black')
    
# корректируем заголовок, оси, подписи осей
plt.title("Зависимость между целями кредита и возвратом кредита в срок\n".upper(), loc='center')
plt.xticks()
plt.yticks()
plt.xlabel("Не вернули кредит, %")
plt.ylabel("Цель кредита")
bar.spines.clear();

**Вывод:** 
Для каждой цели кредита процент возврата равен от 7 до 9.5%. 

Чаще остальных не возвращают кредит в случае цели *Операций с автомобилем*.

Реже остальных не возвращают кредит в случае цели  *Операций с недвижимостью*.

Вероятно такое распределение может быть связано с тем, что:

- автомобили теряют в цене сразу после их покупки, а недвижимость, наоборот, со временем только растет в цене;

- автомобили могут требовать непредвиденных дополнительных расходов (в случае поломок, аварий), непредвиденные расходы в отношении недвижимости случаются, но реже.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 
- технические ошибки:
    - ошибки в коде при обработке данных
    - ошибки при переносе данных
- человеческий фактор:
    - забыли/не захотели заполнить
    - опечатались
    - неверно поняли задачу и т.п.
    - не договорились как нужно заполнять

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

Среднее и медиана различаются, если в выборке присутствуют выбросы (значения, выделяющиеся из общей выборки). 

Медиана позволяет нивелировать их влияние.

### Шаг 4: Общий вывод

1. Была выполнена предобработка данных, а именно исправлены следующие несоответствия: 
- выявлены отрицательные значения в столбце данных о трудовом стаже (days_employed) и приведены к положительным значениям;
- выявлены аномальные значения в столбцу данных о количестве детей (children) и выполнена выборка только строк с корректными значениями;
- выявлены пропуски значений в столбце данных о доходах (total_income) и выполнена замена пропусков на медианные значения;
- выявлены пропуски значений в столбце данных о трудовом стаже (days_employed) и выполнена замена пропусков на медианные значения;
- изменен тип данных (с вещественного на целочисленный) для столбца с данными о доходах (total_income);
- выявлены неявные дубликаты в столбце с данными об образовании (education) и приведены к единому формату;
- удалены дубликаты строк.

2. Выявлены следующие зависимости просрочки:
- заемщики без детей чаще недопускают просрочек по кредиту, в отличие от заемщиков с детьми;
- заемщики, которые никогда не были женаты/замужем, чаще остальных допускают просрочки по кредиту (возможно есть связь с возрастом);
- заемщики с более низким доходом чаще допускают просрочки по кредиту, чем заемщики с более высоким доходом;
- просрочки по кредитам, которые брались с целью операций с автомобилем или получения образования, случаются чаще, чем по кредитам, цель которых была проведение свадьбы или операции с недвижимостью.
    
3. Портреты надежного и рискованного заемщика:
    3.1. Самый надежный заемщик: 
	- без детей;
	- вдовец / вдова;
	- с доходом от 200 000;
	- с целью кредита: операции с недвижимостью.
3.2. Самый ненадежный заемщик:
	- имеет детей;
	- не женат / не замужем или в гражданском браке;
	- с доходом от 50 000 до 200 000;
	- с целью кредита: операции с автомобилем.

4. При принятии решения о выдаче кредита рекомендуется обращать внимание на следующие параметры: 
	- наличие и количество детей;
	- возраст и семейное положение; 
	- уровень дохода;
	- цели кредита.  

Для расчета связей указанных выше параметров между собой требуется проведение дополнительного исслледования. 

